In [9]:
import pandas as pd
import sqlite3

# Read CSV
df = pd.read_csv(r'C:\Users\Kiran\Downloads\uber__1_.csv')

# Create SQLite database (creates file in same folder)
conn = sqlite3.connect('uber_analysis.db')

# Import data (1 line, super fast)
df.to_sql('bookings', conn, if_exists='replace', index=False)

print("✓ Database created: uber_analysis.db")
print(f"✓ Table 'bookings' created with {len(df)} rows")

conn.close()

C:\Users\Kiran\AppData\Local\Temp\ipykernel_23508\1982460662.py:5: DtypeWarning: Columns (9,11,13,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\Kiran\Downloads\uber__1_.csv')


✓ Database created: uber_analysis.db
✓ Table 'bookings' created with 150000 rows


In [10]:
import sqlite3
import pandas as pd

# Connect
conn = sqlite3.connect('uber_analysis.db')

# Query 1: Peak hour analysis
query = """
SELECT 
    CAST(substr(Time, 1, 2) AS INTEGER) as hour_of_day,
    COUNT(*) as total_bookings,
    SUM(CASE WHEN `Booking Status` LIKE '%Cancel%' THEN 1 ELSE 0 END) as cancellations,
    ROUND(SUM(CASE WHEN `Booking Status` LIKE '%Cancel%' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as cancellation_rate
FROM bookings
GROUP BY hour_of_day
ORDER BY cancellation_rate DESC
LIMIT 10
"""

result = pd.read_sql(query, conn)
print(result)

conn.close()

   hour_of_day  total_bookings  cancellations  cancellation_rate
0            5            2786            736              26.42
1           14            7031           1834              26.08
2           15            8202           2106              25.68
3            3            1383            355              25.67
4            4            1321            338              25.59
5           22            5441           1391              25.57
6           19           11047           2820              25.53
7           23            2762            702              25.42
8           18           12397           3146              25.38
9            7            5450           1382              25.36


In [11]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('uber_analysis.db')

query = """
SELECT 
    `Vehicle Type`,
    COUNT(*) as total_bookings,
    SUM(CASE WHEN `Booking Status` = 'Completed' THEN 1 ELSE 0 END) as completed_bookings,
    ROUND(SUM(CASE WHEN `Booking Status` = 'Completed' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as completion_rate,
    SUM(CASE WHEN `Booking Status` = 'Completed' THEN CAST(`Booking Value` AS REAL) ELSE 0 END) as total_revenue,
    ROUND(AVG(CASE WHEN `Booking Status` = 'Completed' THEN CAST(`Booking Value` AS REAL) ELSE NULL END), 2) as avg_booking_value
FROM bookings
GROUP BY `Vehicle Type`
ORDER BY total_revenue DESC
"""

result = pd.read_sql(query, conn)
print("\n=== VEHICLE TYPE PERFORMANCE ===")
print(result)

conn.close()


=== VEHICLE TYPE PERFORMANCE ===
    Vehicle Type  total_bookings  completed_bookings  completion_rate  \
0           Auto           37419               23155            61.88   
1        Go Mini           29806               18549            62.23   
2       Go Sedan           27141               16676            61.44   
3           Bike           22517               14034            62.33   
4  Premier Sedan           18111               11252            62.13   
5          eBike           10557                6551            62.05   
6        Uber XL            4449                2783            62.55   

   total_revenue  avg_booking_value  
0     11727615.0             506.48  
1      9411418.0             507.38  
2      8538560.0             512.03  
3      7144913.0             509.11  
4      5733655.0             509.57  
5      3298157.0             503.46  
6      1406256.0             505.30  


In [12]:
conn = sqlite3.connect('uber_analysis.db')

query = """
SELECT 
    `Pickup Location`,
    `Drop Location`,
    COUNT(*) as trip_count,
    SUM(CAST(`Booking Value` AS REAL)) as total_revenue,
    ROUND(AVG(CAST(`Booking Value` AS REAL)), 2) as avg_revenue
FROM bookings
WHERE `Booking Status` = 'Completed'
GROUP BY `Pickup Location`, `Drop Location`
HAVING COUNT(*) >= 5
ORDER BY total_revenue DESC
LIMIT 20
"""

result = pd.read_sql(query, conn)
print("\n=== TOP 20 ROUTES ===")
print(result)

conn.close()


=== TOP 20 ROUTES ===
        Pickup Location            Drop Location  trip_count  total_revenue  \
0          Nirman Vihar             Vatika Chowk           5         9284.0   
1           Ashok Vihar            Basai Dhankot           9         9280.0   
2      Anand Vihar ISBT          Noida Film City           7         8960.0   
3         Ambience Mall               Akshardham          11         8518.0   
4             Cyber Hub  Gurgaon Railway Station           9         8396.0   
5         Greater Noida                 Jor Bagh           8         8252.0   
6       Noida Extension                 Vaishali           8         8202.0   
7         Greater Noida                  Rithala           7         8082.0   
8           Udyog Vihar            Tagore Garden           9         7940.0   
9             Cyber Hub             Chanakyapuri           7         7900.0   
10             Mehrauli                Hauz Khas           8         7880.0   
11  Udyog Vihar Phase 4      

In [13]:
conn = sqlite3.connect('uber_analysis.db')

query = """
SELECT 
    `Reason for cancelling by Customer` as reason,
    COUNT(*) as count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM bookings WHERE `Booking Status` = 'Cancelled by Customer'), 2) as percentage
FROM bookings
WHERE `Booking Status` = 'Cancelled by Customer'
GROUP BY reason
ORDER BY count DESC
"""

result = pd.read_sql(query, conn)
print("\n=== CUSTOMER CANCELLATION REASONS ===")
print(result)

conn.close()


=== CUSTOMER CANCELLATION REASONS ===
                                         reason  count  percentage
0  Driver is not moving towards pickup location   3990        38.0
1                                 Wrong Address   2520        24.0
2                               Change of plans   1890        18.0
3                        Driver asked to cancel   1470        14.0
4                             AC is not working    630         6.0


In [14]:
conn = sqlite3.connect('uber_analysis.db')

query = """
WITH customer_metrics AS (
    SELECT 
        `Customer ID`,
        COUNT(*) as total_rides,
        SUM(CASE WHEN `Booking Status` = 'Completed' THEN CAST(`Booking Value` AS REAL) ELSE 0 END) as total_revenue
    FROM bookings
    GROUP BY `Customer ID`
)
SELECT 
    CASE 
        WHEN total_rides = 1 THEN 'One-time'
        WHEN total_rides BETWEEN 2 AND 5 THEN 'Occasional'
        WHEN total_rides BETWEEN 6 AND 15 THEN 'Regular'
        ELSE 'VIP'
    END as customer_segment,
    COUNT(*) as customer_count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM customer_metrics), 2) as customer_percentage,
    SUM(total_revenue) as segment_total_revenue,
    ROUND(AVG(total_revenue), 2) as avg_lifetime_value
FROM customer_metrics
GROUP BY customer_segment
ORDER BY segment_total_revenue DESC
"""

result = pd.read_sql(query, conn)
print("\n=== CUSTOMER SEGMENTATION ===")
print(result)

conn.close()


=== CUSTOMER SEGMENTATION ===
  customer_segment  customer_count  customer_percentage  \
0         One-time           82614                79.35   
1       Occasional           20225                19.43   
2          Regular            1274                 1.22   
3              VIP               1                 0.00   

   segment_total_revenue  avg_lifetime_value  
0             23539275.0              284.93  
1             23261702.0             1150.15  
2               458058.0              359.54  
3                 1539.0             1539.00  


In [15]:
conn = sqlite3.connect('uber_analysis.db')

query = """
SELECT 
    CAST(substr(Date, 6, 2) AS INTEGER) as month,
    COUNT(*) as total_bookings,
    SUM(CASE WHEN `Booking Status` = 'Completed' THEN 1 ELSE 0 END) as completed_bookings,
    ROUND(SUM(CASE WHEN `Booking Status` = 'Completed' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as completion_rate,
    SUM(CASE WHEN `Booking Status` = 'Completed' THEN CAST(`Booking Value` AS REAL) ELSE 0 END) as monthly_revenue
FROM bookings
GROUP BY month
ORDER BY month
"""

result = pd.read_sql(query, conn)
print("\n=== MONTHLY TRENDS ===")
print(result)

conn.close()


=== MONTHLY TRENDS ===
    month  total_bookings  completed_bookings  completion_rate  \
0       1           13245                8189            61.83   
1       2           11543                7130            61.77   
2       3           12719                7954            62.54   
3       4           12199                7632            62.56   
4       5           12778                7905            61.86   
5       6           12440                7757            62.36   
6       7           12897                7926            61.46   
7       8           12636                7780            61.57   
8       9           12248                7542            61.58   
9      10           12651                7905            62.49   
10     11           12394                7659            61.80   
11     12           12250                7621            62.21   

    monthly_revenue  
0         4104741.0  
1         3653416.0  
2         4174900.0  
3         3885255.0  
4      

In [16]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('uber_analysis.db')

query = """
SELECT 
    `Payment Method`,
    COUNT(*) as total_transactions,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM bookings WHERE `Booking Status` = 'Completed'), 2) as transaction_percentage,
    SUM(CAST(`Booking Value` AS REAL)) as total_revenue,
    ROUND(SUM(CAST(`Booking Value` AS REAL)) * 100.0 / (SELECT SUM(CAST(`Booking Value` AS REAL)) FROM bookings WHERE `Booking Status` = 'Completed'), 2) as revenue_percentage,
    ROUND(AVG(CAST(`Booking Value` AS REAL)), 2) as avg_transaction_value
FROM bookings
WHERE `Booking Status` = 'Completed'
GROUP BY `Payment Method`
ORDER BY total_revenue DESC
"""

result = pd.read_sql(query, conn)
print("\n=== PAYMENT METHOD ANALYSIS ===")
print(result)

conn.close()


=== PAYMENT METHOD ANALYSIS ===
  Payment Method  total_transactions  transaction_percentage  total_revenue  \
0            UPI               41834                   44.98     21274591.0   
1           Cash               23114                   24.85     11756071.0   
2    Uber Wallet               11206                   12.05      5669070.0   
3    Credit Card                9320                   10.02      4753767.0   
4     Debit Card                7526                    8.09      3807075.0   

   revenue_percentage  avg_transaction_value  
0               45.02                 508.55  
1               24.88                 508.61  
2               12.00                 505.90  
3               10.06                 510.06  
4                8.06                 505.86  


In [17]:
conn = sqlite3.connect('uber_analysis.db')

query = """
SELECT 
    CAST(substr(Time, 1, 2) AS INTEGER) as hour_of_day,
    `Pickup Location`,
    COUNT(*) as total_bookings,
    SUM(CASE WHEN `Booking Status` = 'No Driver Found' THEN 1 ELSE 0 END) as no_driver_bookings,
    ROUND(SUM(CASE WHEN `Booking Status` = 'No Driver Found' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as no_driver_rate,
    ROUND(SUM(CASE WHEN `Booking Status` = 'No Driver Found' THEN 1 ELSE 0 END) * 
          (SELECT AVG(CAST(`Booking Value` AS REAL)) FROM bookings WHERE `Booking Status` = 'Completed'), 0) as estimated_lost_revenue
FROM bookings
WHERE CAST(substr(Time, 1, 2) AS INTEGER) BETWEEN 18 AND 21
GROUP BY hour_of_day, `Pickup Location`
HAVING COUNT(*) >= 20
ORDER BY estimated_lost_revenue DESC
LIMIT 15
"""

result = pd.read_sql(query, conn)
print("\n=== PEAK HOUR SUPPLY-DEMAND ANALYSIS (Top 15 High-Loss Locations) ===")
print(result)

conn.close()


=== PEAK HOUR SUPPLY-DEMAND ANALYSIS (Top 15 High-Loss Locations) ===
    hour_of_day      Pickup Location  total_bookings  no_driver_bookings  \
0            18            Paharganj              77                  13   
1            18          Udyog Vihar              84                  12   
2            20           Vinobapuri              51                  12   
3            18           Badshahpur              80                  11   
4            18        Paschim Vihar              82                  11   
5            18        Saket A Block              74                  11   
6            18         Punjabi Bagh              84                  10   
7            18        Subhash Chowk              86                  10   
8            18          Sushant Lok              91                  10   
9            19     Dwarka Sector 21              69                  10   
10           19              Rithala              77                  10   
11           19  

In [18]:
conn = sqlite3.connect('uber_analysis.db')

query = """
SELECT 
    `Driver Cancellation Reason` as reason,
    COUNT(*) as count,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM bookings WHERE `Booking Status` = 'Cancelled by Driver'), 2) as percentage,
    ROUND(COUNT(*) * (SELECT AVG(CAST(`Booking Value` AS REAL)) FROM bookings WHERE `Booking Status` = 'Completed'), 0) as estimated_revenue_loss
FROM bookings
WHERE `Booking Status` = 'Cancelled by Driver'
GROUP BY reason
ORDER BY count DESC
"""

result = pd.read_sql(query, conn)
print("\n=== DRIVER CANCELLATION REASONS ===")
print(result)

conn.close()


=== DRIVER CANCELLATION REASONS ===
                                reason  count  percentage  \
0               Customer related issue  10800        40.0   
1        Personal & Car related issues   7020        26.0   
2       The customer was coughing/sick   5400        20.0   
3  More than permitted people in there   3780        14.0   

   estimated_revenue_loss  
0               5488325.0  
1               3567411.0  
2               2744162.0  
3               1920914.0  


In [19]:
conn = sqlite3.connect('uber_analysis.db')

query = """
SELECT 
    CASE CAST(strftime('%w', Date) AS INTEGER)
        WHEN 0 THEN 'Sunday'
        WHEN 1 THEN 'Monday'
        WHEN 2 THEN 'Tuesday'
        WHEN 3 THEN 'Wednesday'
        WHEN 4 THEN 'Thursday'
        WHEN 5 THEN 'Friday'
        WHEN 6 THEN 'Saturday'
    END as day_of_week,
    COUNT(*) as total_bookings,
    SUM(CASE WHEN `Booking Status` = 'Completed' THEN 1 ELSE 0 END) as completed_bookings,
    ROUND(SUM(CASE WHEN `Booking Status` = 'Completed' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as completion_rate,
    SUM(CASE WHEN `Booking Status` = 'Completed' THEN CAST(`Booking Value` AS REAL) ELSE 0 END) as daily_revenue,
    ROUND(AVG(CASE WHEN `Booking Status` = 'Completed' THEN CAST(`Booking Value` AS REAL) ELSE NULL END), 2) as avg_booking_value
FROM bookings
GROUP BY day_of_week
ORDER BY 
    CASE day_of_week
        WHEN 'Monday' THEN 1
        WHEN 'Tuesday' THEN 2
        WHEN 'Wednesday' THEN 3
        WHEN 'Thursday' THEN 4
        WHEN 'Friday' THEN 5
        WHEN 'Saturday' THEN 6
        WHEN 'Sunday' THEN 7
    END
"""

result = pd.read_sql(query, conn)
print("\n=== DAY OF WEEK PERFORMANCE ===")
print(result)

conn.close()


=== DAY OF WEEK PERFORMANCE ===
  day_of_week  total_bookings  completed_bookings  completion_rate  \
0      Monday           21274               13215            62.12   
1     Tuesday           21746               13381            61.53   
2   Wednesday           21337               13165            61.70   
3    Thursday           21526               13343            61.99   
4      Friday           21100               13125            62.20   
5    Saturday           21384               13231            61.87   
6      Sunday           21633               13540            62.59   

   daily_revenue  avg_booking_value  
0      8699700.0             658.32  
1      6282103.0             469.48  
2      5866192.0             445.59  
3      5994406.0             449.25  
4      5980247.0             455.64  
5      5994234.0             453.04  
6      8443692.0             623.61  
